<a href="https://www.kaggle.com/code/andrejzuba/h-m-recommender-quick-and-easy-model-build?scriptVersionId=110772205" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## First of all I loaded all the data into Snowflake database for easier manipulation
I made a simple datamart with snap dates '2020-09-01', '2020-09-08' for training
and '2020-09-15' for testing, datamart was exported in one csv file named
model_build_base.csv

#### The train set contains following data:
* 495,774 rows of SOLD=1 -> All transactions that were sold in week '2020-09-02' to '2020-09-08' and week '2020-09-09' to '2020-09-15'
* 4,000,000 randomly chosen combinations of customer_id and article_id that were not sold with SOLD=0
* All article information, customer information and customer transaction history info is also joined

#### The test set contains following data:
* Similar to train set just shifted forward for week between '2020-09-15' and '2020-09-22'

#### Predictors
Predictors were basic aggregations like number bought last month or days since 
last purchase on three different levels:
* CUST - on customer level, here I included also number of purchases across different product groups
* ART - on article (item) level
* CUSTART - combination of customer and article, last purchase, number purchased last month etc.


#### Fitting
A simple XGBoost model was fitted


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#importing custom library
!pip install git+https://github.com/Vrboska/mofr@master

In [ ]:
import pandas as pd
import numpy as np
import random
#import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_tree
import math

import xgboost as xgb

In [ ]:
seed=1234

In [ ]:
df=pd.read_csv('/kaggle/input/hm-model-build-base/model_build_base.csv')#.sample(100000)

In [ ]:
#filling in nulls for CUSTART predictors
df[['CUSTART_QUANTITY_SOLD_1M', 'CUSTART_QUANTITY_SOLD_3M', 'CUSTART_QUANTITY_SOLD_12M', 'CUSTART_QUANTITY_SOLD_OVERALL', 'CUSTART_NUM_CHANNEL_2', 'ART_AVERAGE_PRICE', 'ART_NUM_CHANNEL_2']]=df[['CUSTART_QUANTITY_SOLD_1M', 'CUSTART_QUANTITY_SOLD_3M', 'CUSTART_QUANTITY_SOLD_12M', 'CUSTART_QUANTITY_SOLD_OVERALL', 'CUSTART_NUM_CHANNEL_2', 'ART_AVERAGE_PRICE', 'ART_NUM_CHANNEL_2']].fillna(value=0)

In [ ]:
df=df[df['ART_QUANTITY_SOLD_1M']>0]

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
train_mask=(df['SNAP_DATE']=='2020-09-08')|(df['SNAP_DATE']=='2020-09-01')#|
valid_mask=df['SNAP_DATE']=='2020-09-15'

In [ ]:
df[train_mask]['SOLD'].value_counts()

In [ ]:
df[valid_mask]['SOLD'].value_counts()

# Data Exploration

In [ ]:
df[train_mask].describe()

In [ ]:
df[train_mask].describe(include=['O'])

# Data transformations

In [ ]:
col_target='SOLD'
col_exclude=[
'SNAP_DATE',
'CUSTOMER_ID',
'ARTICLE_ID',
# 'ART_DAYS_SINCE_FIRST_PURCHASE',
# 'ART_DAYS_SINCE_LAST_PURCHASE',

col_target,
    

]#+[col for col in df.columns if 'CUSTART' in col]

In [ ]:
col_exclude

## Categorical transformations

In [ ]:
import category_encoders as ce

In [ ]:
# potential predictors without encoding
cat_preds = [col for col in df.select_dtypes(include="object") if col not in col_exclude]
bool_preds = [col for col in df.select_dtypes(include="bool") if col not in col_exclude]
datetime_preds = [col for col in df.select_dtypes(include="datetime") if col not in col_exclude]

In [ ]:
cat_preds

### Target Encoding

In [ ]:
# # bayesian target encoding
encoder = ce.TargetEncoder(min_samples_leaf=1, smoothing=1.0)
encoder.fit_transform(df[train_mask][cat_preds], df[train_mask][col_target])

df = pd.concat([df, encoder.transform(df[cat_preds]).add_prefix("BAYES_")], axis=1)

In [ ]:
col_preds=[col for col in df.select_dtypes(include=["int","float"]) if col not in col_exclude]

In [ ]:
len(col_preds)

# Fitting models

In [ ]:
(df[train_mask][col_target]>0).value_counts(normalize=True)

## XGBoost model

## Fitting model

In [ ]:
xgb_model = XGBClassifier(max_depth=4, seed=seed, colsample_bytree=0.5, gamma=1, min_child_weight=5, n_estimators=100)
xgb_model.fit(df[train_mask].loc[:, col_preds], df[train_mask][col_target], verbose=0, eval_metric='logloss')

In [ ]:
df['XGB_SCORE']=xgb_model.predict_proba(df[col_preds])[:, 1]

In [ ]:
print('The Lift on the train set is: '+ str(mofr.metrics.lift(df[train_mask][col_target], df[train_mask]['XGB_SCORE'])))
print('The gini on the train set is: '+ str(mofr.metrics.gini(df[train_mask][col_target], df[train_mask]['XGB_SCORE'])))
print('The accuracy on the train set is: '+ str(mofr.metrics.accuracy_score(df[train_mask][col_target], df[train_mask]['XGB_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')
print('The Lift on the valid set is: '+ str(mofr.metrics.lift(df[valid_mask][col_target], df[valid_mask]['XGB_SCORE'])))
print('The gini on the valid set is: '+ str(mofr.metrics.gini(df[valid_mask][col_target], df[valid_mask]['XGB_SCORE'])))
print('The accuracy on the valid set is: '+ str(mofr.metrics.accuracy_score(df[valid_mask][col_target], df[valid_mask]['XGB_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')

random_customers=pd.DataFrame(np.random.choice(df[valid_mask]['CUSTOMER_ID'].unique(), size=1000))
random_customers=random_customers.rename(columns={0:'CUSTOMER_ID'})
top12=df[(valid_mask)].merge(random_customers)[['SOLD','CUSTOMER_ID', 'ARTICLE_ID', 'XGB_SCORE']].groupby('CUSTOMER_ID').apply(lambda x : x.sort_values(by = 'XGB_SCORE', ascending = False).head(12).reset_index(drop = True)).reset_index(drop = True)
precision=pd.DataFrame(top12.groupby('CUSTOMER_ID')['SOLD'].apply(np.mean)).rename(columns={'SOLD':'PRECISION'}).reset_index()
mean_precision=np.mean(precision['PRECISION'])
print(f'The mean precision on valid set is: {mean_precision}')

In [ ]:
from mofr.basic_evaluators.ROCCurve import ROCCurveEvaluator
df['one']=1

rce=ROCCurveEvaluator()
rce.d(df[valid_mask]).t([(col_target,'one')]).s(['XGB_SCORE'])
rce.get_graph()

del df['one']

In [ ]:
sorted_idx = xgb_model.feature_importances_.argsort()
order_ = []
for i in sorted_idx:
  order_.append(col_preds[i])
plt.figure(figsize=(10, 10))
fig = plt.barh(order_, xgb_model.feature_importances_[sorted_idx])
plt.xlabel("Xgboost Feature Importance")
plt.show()

In [ ]:
results=[]
for col in col_preds:
    results.append((col, np.abs(mofr.metrics.gini(df[valid_mask][col_target], df[valid_mask][col]))))
  
pd.DataFrame(results, columns=['Predictor', 'GINI']).sort_values(by='GINI', ascending=False)[0:30]

### Saving the model

In [ ]:
import pickle
file_name = "hm_xgb_model.pkl"

# save
pickle.dump(xgb_model, open(file_name, "wb"))

# # load
# #xgb_model= pickle.load(open(file_name, "rb"))

In [ ]:
import pickle
file_name = "hm_encoder.pkl"

# save
pickle.dump(encoder, open(file_name, "wb"))

# # load
# #encoder = pickle.load(open(file_name, "rb"))

## SHAP values

In [ ]:
import shap  # package used to calculate Shap values

In [ ]:
row_to_show = 1
data_for_prediction = df[train_mask][col_preds].iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

# Create object that can calculate shap values
explainer = shap.TreeExplainer(xgb_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction_array)
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, data_for_prediction)

In [ ]:
shap_values = explainer.shap_values(df[train_mask][col_preds])
shap.summary_plot(shap_values, df[train_mask][col_preds])

In [ ]:
shap.dependence_plot('ART_AVERAGE_PRICE', shap_values, df[train_mask][col_preds], interaction_index="AGE")

# Future predictions part

In [ ]:
del df

In [ ]:
import pickle
file_name1= "/kaggle/input/models/hm_xgb_model_custart_full.pkl"
file_name2= "/kaggle/input/models/hm_encoder_custart_full.pkl"

# load
xgb_model= pickle.load(open(file_name1, "rb"))
encoder= pickle.load(open(file_name2, "rb"))

In [ ]:
import pandas as pd

In [ ]:
articles=pd.read_csv('/kaggle/input/hm-model-build-base/articles_predictions.csv')#.fillna(999)
customers=pd.read_csv('/kaggle/input/hm-model-build-base/customers_prediction.csv').fillna(0)

In [ ]:
del customers['Unnamed: 0']

In [ ]:
customers['CUSTOMER_ID10']=customers['CUSTOMER_ID'].apply(lambda x: x[0:10])

In [ ]:
articles['ARTICLE_ID6']=articles['ARTICLE_ID'].apply(lambda x: int(str(x)[0:6]))

## Scoring customers in batches to produce submission file
For each customer a simple logic was used to generate up to 500 candidate items.
The logic was using the following blend of products:
* Top 100 products bought by the customer last month
* Top 100 products bought by the customer overall
* Top 100 products bought by all customers last month
* Top 100 products bought by all customers last month
* top 100 products for top PRODUCT_TYPE_NAME of the customer

Combination of customers and suggested candidate items were stored in dataset 'model_suggested_items_enriched_sorted.csv'; this file is also enriched with the CUSTART predictors (I put only small sample of this dataset here);

At the end this approach was able to produce a solid score of 0.02251 which is only a little short of bronze medals. This was without using any special computational environment. The biggest difference made was by adding CUSTART predictors (on the level of customer and article), which pushed the performance from 0.007 area to close to bronze medals. 

It was a pleasure solving this competition and amazing way to learn about recommender systems. Thanks to Kaggle and H&M for this opportunity.

In [ ]:
submission=pd.DataFrame()

In [ ]:
def process_chunk(chunk):
    chunk=chunk.rename(columns={'CUSTOMER_ID':'CUSTOMER_ID10'}).drop_duplicates()
    chunk.drop_duplicates(subset=['CUSTOMER_ID10', 'ARTICLE_ID'], inplace=True)
    chunk=chunk.merge(articles, how='left', on='ARTICLE_ID')
    chunk=chunk.merge(customers, how='left', left_on='CUSTOMER_ID10', right_on='CUSTOMER_ID10')
    chunk = pd.concat([chunk, encoder.transform(chunk[cat_preds]).add_prefix("BAYES_")], axis=1)
    
    chunk['XGB_SCORE']=xgb_model.predict_proba(chunk[xgb_model.feature_names_in_])[:, 1]
    chunk['ARTICLE_ID']=chunk['ARTICLE_ID'].apply(str).apply(lambda x: x.zfill(10))
    a=chunk[['CUSTOMER_ID', 'ARTICLE_ID', 'XGB_SCORE']].groupby('CUSTOMER_ID').apply(lambda x : x.sort_values(by = 'XGB_SCORE', ascending = False).head(12).reset_index(drop = True)).reset_index(drop = True)
    b=pd.DataFrame(a.groupby('CUSTOMER_ID')['ARTICLE_ID'].apply(list).apply(' '.join)).reset_index(drop=False).rename(columns={'ARTICLE_ID':'PREDICTION'})
    return b

In [ ]:
n=0
chunksize = 10 ** 6
with pd.read_csv('/kaggle/input/suggested-items/model_suggested_items_enriched_ordered.csv', chunksize=chunksize) as reader:
    for chunk in reader:
        print(f'{n}: {round(n/3.94,2)} % done')
        submission=pd.concat([submission,process_chunk(chunk)])
        n+=1
        break

In [ ]:
len(submission)

In [ ]:
submission.drop_duplicates(subset='CUSTOMER_ID',keep='first', inplace=True, ignore_index=False)

In [ ]:
submission.to_csv('submission.csv', index=False)